Check the status of tube lines, overground, DLR, Elizabeth line and tram
https://api.tfl.gov.uk/line/mode/tube,overground,dlr,elizabeth-line,tram/status

29-6 added overground state

In [1]:
import requests
import json
import pprint

In [2]:
import time

In [3]:
from datetime import datetime

In [4]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2024-12-12 15:28:56'

In [5]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [6]:
# List of allowed lines
lines = ['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Elizabeth line',
 'Hammersmith & City',
 'Jubilee',
 'Liberty',
 'Lioness',
 'Metropolitan',
 'Mildmay',
 'Northern',
 'Piccadilly',
 'Suffragette',
 'Tram',
 'Victoria',
 'Waterloo & City',
 'Weaver',
 'Windrush']

In [7]:
ls

development.ipynb


In [8]:
len(lines)

20

# Basic function of API

In [9]:
#url = 'https://api.tfl.gov.uk/line/mode/tube/status'   # tube only
url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,elizabeth-line,tram/status'
response = requests.get(url).json() # get the response object

In [10]:
print_json(response)

[{'$type': 'Tfl.Api.Presentation.Entities.Line, Tfl.Api.Presentation.Entities',
  'created': '2024-12-10T18:12:52.143Z',
  'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, '
                        'Tfl.Api.Presentation.Entities'},
  'disruptions': [],
  'id': 'bakerloo',
  'lineStatuses': [{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, '
                             'Tfl.Api.Presentation.Entities',
                    'created': '0001-01-01T00:00:00',
                    'id': 0,
                    'statusSeverity': 10,
                    'statusSeverityDescription': 'Good Service',
                    'validityPeriods': []}],
  'modeName': 'tube',
  'modified': '2024-12-10T18:12:52.143Z',
  'name': 'Bakerloo',
  'routeSections': [],
  'serviceTypes': [{'$type': 'Tfl.Api.Presentation.Entities.LineServiceTypeInfo, '
                             'Tfl.Api.Presentation.Entities',
                    'name': 'Regular',
                    'uri': '/Line/Route?ids=Baker

In [11]:
lines = [line['name'] for line in response]
lines

['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Elizabeth line',
 'Hammersmith & City',
 'Jubilee',
 'Liberty',
 'Lioness',
 'Metropolitan',
 'Mildmay',
 'Northern',
 'Piccadilly',
 'Suffragette',
 'Tram',
 'Victoria',
 'Waterloo & City',
 'Weaver',
 'Windrush']

In [12]:
data_dict = dict.fromkeys(lines)    # Create the empty dict for the data
data_dict

{'Bakerloo': None,
 'Central': None,
 'Circle': None,
 'District': None,
 'DLR': None,
 'Elizabeth line': None,
 'Hammersmith & City': None,
 'Jubilee': None,
 'Liberty': None,
 'Lioness': None,
 'Metropolitan': None,
 'Mildmay': None,
 'Northern': None,
 'Piccadilly': None,
 'Suffragette': None,
 'Tram': None,
 'Victoria': None,
 'Waterloo & City': None,
 'Weaver': None,
 'Windrush': None}

Main logic for parsing the data into a useful dict

In [13]:
data_dict = {}

for line in response:                     # Assign data via key
    try:
        statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
        state = ' + '.join(sorted(set(statuses)))
    
        if state == 'Good Service':   # if good status, this is the only status returned
            reason =  'Nothing to report'
        else:
            reason = ' *** '.join([status['reason'] for status in line['lineStatuses']])

        data_dict[line['name']] = {'State': state, 'Description': reason}
        
    except:
        data_dict[line['name']] = {'State': None, 'Description': 'Error parsing API data'}
    
print_json(data_dict)

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'DLR': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'District': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Elizabeth line': {'Description': 'Nothing to report',
                    'State': 'Good Service'},
 'Hammersmith & City': {'Description': 'Nothing to report',
                        'State': 'Good Service'},
 'Jubilee': {'Description': 'Jubilee Line: Minor delays between Stratford and '
                            'North Greenwich while we fix a faulty train. GOOD '
                            'SERVICE on the rest of the line. ',
             'State': 'Minor Delays'},
 'Liberty': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Lioness': {'Description': 'Nothing to report', 'State': 'Good Service'},
 